**Step-1: Install the packages**

In [1]:
pip install langchain langchain-groq langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 6.7 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [2]:
pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 4.2 MB/s eta 0:00:00


In [4]:
pip install chromadb

**Step-2: Retrive LLM model using groq**

In [ ]:
# Groq is just one provider that gives you access to LLMs
from langchain_groq import ChatGroq
groq_api="GROQ API KEY"

llm = ChatGroq(
    api_key=groq_api,
    model="llama-3.3-70b-versatile",
    temperature=0.7,
    max_tokens=512
)
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7c0a3b6303e0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7c0a3b5a2840>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'), max_tokens=512)

**Step-3: Load the pdf**

In [6]:
from langchain.document_loaders import PyPDFLoader
# from docx import Document
# read from the data source : pdf
#pdf_path = "/content/nareshiT_context2.pdf"
pdf_path = "/content/pythonlearn.pdf"

# load the pdf
loader = PyPDFLoader(pdf_path)
documents = loader.load()

**Step-4: Split into Text**

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
# ------------------ SPLIT ------------------
# in real world, we would fine-tune the model by adjusting the chunk_size and chunk_overlap values
splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=200
)
docs = splitter.split_documents(documents)
print(docs)

[Document(metadata={'producer': 'dvips + GPL Ghostscript 9.18', 'creator': 'LaTeX with hyperref package', 'creationdate': '2022-04-07T00:56:29-04:00', 'moddate': '2022-04-07T00:56:29-04:00', 'title': 'Python for Everybody', 'subject': '', 'author': 'Exploring Data Using Python 3', 'keywords': '', 'source': '/content/pythonlearn.pdf', 'total_pages': 249, 'page': 0, 'page_label': 'i'}, page_content='Python for Everybody\nExploring Data Using Python 3\nDr. Charles R. Severance'), Document(metadata={'producer': 'dvips + GPL Ghostscript 9.18', 'creator': 'LaTeX with hyperref package', 'creationdate': '2022-04-07T00:56:29-04:00', 'moddate': '2022-04-07T00:56:29-04:00', 'title': 'Python for Everybody', 'subject': '', 'author': 'Exploring Data Using Python 3', 'keywords': '', 'source': '/content/pythonlearn.pdf', 'total_pages': 249, 'page': 1, 'page_label': 'ii'}, page_content='Credits\nEditorial Support: Elliott Hauser, Sue Blumenberg\nCover Design: Aimee Andrion\nPrinting History\n• 2016-Jul

**Step-5: Divide text into Embeddings and store in vector database**


In [8]:
# ------------------ EMBEDDINGS + VECTORSTORE ------------------
# use the llm model to convert to embeddings
# use the chroma db to store the vectors
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

vectordb = Chroma.from_documents(
    documents=docs,
    embedding=embeddings,
)
vectordb.persist()

# give top 10 chunks in output
retriever = vectordb.as_retriever(search_kwargs={"k": 10})

/tmp/ipython-input-2377540335.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.w

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/tmp/ipython-input-2377540335.py:12: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


**Step-6: Develop a Prompt**

In [9]:
# ------------------ CUSTOM PROMPT ------------------
system_prompt = """
Info from the Book Python Learn textbook
"""

**Step-7: Create a PromptTemplate**

In [10]:
from langchain.prompts import PromptTemplate
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate(
    template="""
    Answer as the book Python Learn chatbot
CONTEXT:
{context}

QUESTION:
{question}


""",
    input_variables=["context", "question"]
)


**Step-8: Build Retrieval QA**

In [11]:
from langchain.chains import RetrievalQA
# llm model will get the top 10 answers from vector database and combine it with the prompt
retrieval_qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt_template}
)


**Step-9: Ask the Question**

In [12]:
query = "What is Python latest version"
response = retrieval_qa.run(query)

print("\n--- Model Answer ---\n")
print(response)

# drawback : this lacks the specific and accurate information

/tmp/ipython-input-4215790006.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = retrieval_qa.run(query)



--- Model Answer ---

The latest version of Python is not explicitly mentioned in the provided text. However, it does mention Python 3.5.1 (v3.5.1:37a07cee5969, Dec 6 2015, 01:54:25) as the version used in an example. To find the latest version of Python, I recommend checking the official Python website at www.python.org for the most up-to-date information.


### It is giving information only from document, this is called standard RAG

### If we connect any API calls then fetch the data, this indicates agentic behaviour

## **Agentic** **RAG**

**Step-10: Fetch data from URL**

In [21]:
from bs4 import BeautifulSoup
import requests

def scrape_python_updates(query: str):
  print("======")

  # URL of official Python downloads page
  url = "https://www.python.org/downloads/"

  # Fetch the HTML content
  response = requests.get(url)
  response.raise_for_status()  # raise error if request failed

  # Parse with BeautifulSoup
  soup = BeautifulSoup(response.text, "html.parser")

  # Find all release versions
  releases = soup.find_all("span", class_="release-number")

  # Extract context specific text
  python_releases = [release.get_text(strip=True) for release in releases]

  list_python_releases=[]
  # Print result
  print("Python Releases:")
  for r in python_releases:
    list_python_releases.append(r)
  if not list_python_releases:
    return f"No information found for '{query} on Python official website"
  return list_python_releases[:5]


**Step-11: Course Schedule and Trainer Fetcher Tool Definition**

In [29]:
## Creating one agent : Python latest versions

from langchain.agents import initialize_agent, Tool, AgentType
webScrapper_tool = Tool(
    name="Python latest versions",
    func=scrape_python_updates,
    description=(
        "Use this tool to get the latest release of Python version information."
        "Input can be Python name to look latest versions."
    )
)


**Step-12: Defining a Document Retrieval Tool**

In [30]:
## Creating second agent : retrives data from db

retriever_tool = Tool(
    name="Document Retriever",
    func=lambda q: retrieval_qa.run(q),
    description="Answer queries using Python textbook pdf documents."
)

**Step-13: Initialize the agent**

In [31]:
from re import VERBOSE
tools = [retriever_tool, webScrapper_tool]

# Initialize agent with a more flexible agent type

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

**Step-14: Provide instructions to the agent**

In [32]:
system_instruction = """
You are a helpful adminstrator to provide Python information
- If user asks Python information, first try the Document Retriever tool (PDF).
- If no relevant info is found, use the Pyhton Web Scraper tool.
- Always provide a friendly, student-facing final answer.
"""

**Step-15: Start query**

In [33]:
query = "Tell me about latest Python release versions"
response = agent.run(f"{system_instruction}\n\nQuestion: {query}")

print("\n--- Admin Answer ---\n", response)



> Entering new AgentExecutor chain...
Thought: The user is asking for information about the latest Python release versions. To find this information, I should first try to retrieve it from the Python textbook PDF documents using the Document Retriever tool.

Action:
```
{
  "action": "Document Retriever",
  "action_input": "latest Python release versions"
}
```


Observation: Hello, I'm the Python Learn chatbot. You're looking for the latest Python release versions. According to the text, the versions mentioned are:

* Python 3.5.1 (v3.5.1:37a07cee5969, Dec 6 2015, 01:54:25)
* Python 3.8 (mentioned in the documentation links)
* Python 3.7 (mentioned in the documentation links)
* Python 3.5 (mentioned in the documentation links)
* Python 2.7 (mentioned in the documentation links)

Please note that these versions may not be the latest ones, as the text is from a book and might not be up-to-date. For the latest Python release versions, I recommend checking the official Python website: h

## Spot ON-1

- Difference between RecursiveCharacterTextSplitter vs CharacterTextSplitter

**1. Character-level splitting**

Splits text into chunks of fixed character length (e.g., 500 characters per chunk).

- Pros:

    - Simple to implement.

- Cons:

    - Can cut sentences in half, losing semantic meaning.

    - Chunks might be arbitrary, making embeddings less semantically meaningful.

**2. Recursive splitting (hierarchical)**

Splits text intelligently, e.g.:

    - First by document → sections

    - Then paragraphs

    - Then sentences, if needed.


- Pros:

    - Preserves semantic units (sentences/paragraphs).

    - Embeddings capture meaningful context, improving RAG performance.

- Cons:

    - Slightly more complex to implement.
